In [150]:
import time
import pandas as pd
#라이브러리 불러오기
from selenium import webdriver as wb
# 웹브라우저를 제어하기 위한 모듈
from selenium.webdriver.common.by import By
# 선택자를 찾기 위한 모듈
from selenium.webdriver.common.keys import Keys #웹에 값을 입력하기 위한 모듈
from selenium.webdriver.chrome.service import Service
# 사용자가 사용하는 최신버전의 크롬 드라이버와 일치하는 드라이버를 찾아주는 모듈
from webdriver_manager.chrome import ChromeDriverManager
# 최신버전의 크롬 드라이버를 찾고 다운로드 해주는 모듈
from bs4 import BeautifulSoup as bs
from IPython.display import display

In [142]:
press_df = pd.read_csv('./data/언론사리스트.csv')
display(press_df)

,press_name,press_href,title,content
0,조선일보,https://www.chosun.com,h1.article-header__headline>span,p.article-body__content
1,연합뉴스,https://www.yna.co.kr,h1.tit01,div.story-news>p
2,SBS,https://news.sbs.co.kr,h1.article_main_tit,div.text_area
3,SBS Biz,https://biz.sbs.co.kr,h3.ah_big_title,div.ab_text
4,YTN,https://www.ytn.co.kr,h2.news_title>span,div.paragraph>span
5,뉴시스,https://www.newsis.com,h1.tit,div.viewer>article
6,중앙일보,https://www.joongang.co.kr,h1.headline,div.article_body>p
7,아시아경제,https://view.asiae.co.kr,div.area_title>h1,div.fb-quotable>p
8,이데일리,https://www.edaily.co.kr,div.news_titles>h1,div.news_body
9,채널A,https://ichannela.com,div.news_article_title>h2>strong,div.news_page_txt


In [143]:
driver = wb.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.naver.com")

# 검색어 입력
search_box = driver.find_element(By.ID, "query")
search_box.send_keys("트럼프")
search_box.send_keys(Keys.ENTER)
time.sleep(2)

# 뉴스 버튼 클릭
news_btn = driver.find_element(By.XPATH, "//a[starts-with(@href, '?ssc=tab.news')]")
news_btn.click()

# 스크롤 다운
news_links = []
while True:
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    news_links = driver.find_elements(By.CLASS_NAME, "bXkY_j9rDdvFawUQ5HBd.UpDjg8Q2DzdaIi4sfrjX")
    if(len(news_links) < 100):
        time.sleep(1)
        continue
    break

driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.HOME)
time.sleep(1)

press_list: list[str] = []
title_list: list[str] = []
content_list: list[str] = []
uri_list: list[str] = []
unknown_press_list: list[str] = []

# 뉴스 기사 정보 수집
for i, link in enumerate(news_links):
    link.click()
    time.sleep(1)
    driver.switch_to.window(driver.window_handles[-1])
    current_press_row = press_df[press_df['press_href'].apply(lambda href: href in driver.current_url)]
    cur_press = '언론사 정보 없음'
    if not current_press_row.empty:
        try:
            cur_press = str(current_press_row['press_name'].values[0])
            soup = bs(driver.page_source, 'lxml')
            current_title = soup.select(current_press_row['title'].values[0])
            selected_content = soup.select(current_press_row['content'].values[0])
            
            current_content = ''
            for content in selected_content:
                current_content += str(content.text).strip()
            current_content = str(current_content).replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').strip()
            
            press_list.append(cur_press)
            title_list.append(str(current_title[0].text).replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').strip())
            content_list.append(current_content)
            uri_list.append(driver.current_url)
        except Exception as e:
            print(f"오류 발생: 건너뜀. 언론사: {cur_press} | 오류 내용: {e}")
    else:
        unknown_press_list.append(f"https://{driver.current_url.split('://')[1].split('/')[0]}")

    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    print(f"\r                                                                                     ", end='')
    print(f"\r진행 중: {i + 1}/{len(news_links)} - 언론사: {cur_press} | 수집된 기사: {len(press_list)}", end='')
    time.sleep(1)


driver.close()
time.sleep(2)
driver.quit()

print("\n수집 완료!")
print("=" * 50)
print("언론사 정보 없음\n")
for url in unknown_press_list:
    print(url)

진행 중: 100/100 - 언론사: 중앙일보 | 수집된 기사: 96                                               
수집 완료!
언론사 정보 없음

https://mb.hansung.co.kr
https://www.kmib.co.kr
https://www.segye.com
https://www.mbn.co.kr


In [149]:
news_df = pd.DataFrame({
    'press_name': press_list,
    # 'uri': uri_list,
    'title': title_list,
    'content': content_list
})
display(news_df)
news_df.to_csv('./data/뉴스기사.csv', index=False, encoding='utf-8-sig')

,press_name,title,content
0,연합뉴스,"트럼프 ""푸틴-젤렌스키 회담 조율 시작…이어 나 포함 3자회담""(종합)",(워싱턴=연합뉴스) 조준형 특파원 = 도널드 트럼프 미국 대통령이 우크라이나전쟁의 ...
1,조선일보,"유럽 정상들 만난 트럼프 ""영토 교환 결정, 젤렌스키가 내려야""",도널드 트럼프 미국 대통령이 18일 백악관에서 유럽 정상들을 만나 “푸틴 러시아 대...
2,이데일리,"러 크렘린궁 ""푸틴-트럼프, 우크라 협상 대표 격상 논의""",[이데일리 방성훈 기자] 러시아 크렘린궁(대통령실)은 18일(현지시간) 블라디미르 ...
3,TV조선,"푸틴 최측근 ""아부 쏟아낸 유럽, 트럼프 압도 못해""",블라디미르 푸틴 러시아 대통령의 측근으로 꼽히는 드미트리 메드베데프 러시아 국가안보...
4,연합뉴스,"전 CIA 국장 ""푸틴, 트럼프 전용차에 칩 부착했을 수도""",(서울=연합뉴스) 고일환 기자 = 블라디미르 푸틴 러시아 대통령이 도널드 트럼프 미...
...,...,...,...
91,한국일보,"트럼프, 우크라 '미군 주둔 불가' 입장 재확인… ""유럽이 병력 보낼 것""",도널드 트럼프 미국 대통령이 우크라이나 내 미군 주둔 가능성을 다시금 부정했다. 다...
92,연합뉴스,"멜라니아 트럼프, 푸틴에 평화 호소…""아이들 웃음 되찾아달라""",(워싱턴=연합뉴스) 김동현 특파원 = 도널드 트럼프 미국 대통령의 부인인 멜라니아 ...
93,서울신문,"톰 크루즈, 트럼프 시상하는 ‘케네디 공로상’ 거절",이미지 확대 ...
94,TV조선,"""트럼프·푸틴 정상회담서 '바이든 혐오' 의견일치""",도널드 트럼프 미국 대통령과 블라디미르 푸틴 러시아 대통령이 정상회담에서 휴전 합의...
